# Пример на реальных данных

In [1]:
import pandas as pd
import numpy as np

У нас есть 3 csv файла. Первый хранит в себе данные по штатам Америки и численности населения, причем численность дана с 2013 по 1990 годы для людей до 18 лет и для всех. Второй представляет информацию также о штате (причем здесь уже не аббревиатура штата, а полное название) и его площади. Третий хранит полное название штата и аббревиатуру. Попробуем объединить все наборы данных. 

In [2]:
# подгружаем каждый из файлов и смотрим на первые 5 наблюдений
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display(pop.head(), areas.head(), abbrevs.head())

state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

In [3]:
merged = pd.merge(pop, abbrevs, how = 'outer', 
                 left_on ='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis = 1)
merged.head()

state/region     ages  year  population    state
0           AL  under18  2012   1117489.0  Alabama
1           AL    total  2012   4817528.0  Alabama
2           AL  under18  2010   1130966.0  Alabama
3           AL    total  2010   4785570.0  Alabama
4           AL  under18  2011   1125763.0  Alabama

Посмотрим, не пропустили ли мы какую-нибудь информацию

In [4]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Видим, что в столбце населения и штата есть пропущенные значения. Можно сделать вывод, что DF-1 хранил в себе большее кол-во штатов

In [5]:
# смотрим на первые 5 наблюдений
merged[merged['population'].isnull()].head()

state/region     ages  year  population state
2448           PR  under18  1990         NaN   NaN
2449           PR    total  1990         NaN   NaN
2450           PR    total  1991         NaN   NaN
2451           PR  under18  1991         NaN   NaN
2452           PR    total  1993         NaN   NaN

Действительно, штата PR видимо не было в DF-3. Посмотрим, какие штаты еще пропущены

In [6]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Назовем сами пропущенные строки в колонке "state"

In [7]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Теперь соединим последний df

In [33]:
final = pd.merge(merged, areas, on ='state', how='left')
final.head()

state/region     ages  year  population    state  area (sq. mi)
0           AL  under18  2012   1117489.0  Alabama        52423.0
1           AL    total  2012   4817528.0  Alabama        52423.0
2           AL  under18  2010   1130966.0  Alabama        52423.0
3           AL    total  2010   4785570.0  Alabama        52423.0
4           AL  under18  2011   1125763.0  Alabama        52423.0

In [34]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [35]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Выкинем все NA

In [36]:
final.dropna(inplace=True)
final.tail()

state/region     ages  year  population        state  area (sq. mi)
2491           PR  under18  2010    896945.0  Puerto Rico         3515.0
2492           PR  under18  2011    869327.0  Puerto Rico         3515.0
2493           PR    total  2011   3686580.0  Puerto Rico         3515.0
2494           PR  under18  2012    841740.0  Puerto Rico         3515.0
2495           PR    total  2012   3651545.0  Puerto Rico         3515.0

А теперь найдем плотность населения

In [37]:
final2012 = final.query("year == 2012 & ages == 'total'")
final2012.head()

state/region   ages  year  population       state  area (sq. mi)
1             AL  total  2012   4817528.0     Alabama        52423.0
95            AK  total  2012    730307.0      Alaska       656425.0
97            AZ  total  2012   6551149.0     Arizona       114006.0
191           AR  total  2012   2949828.0    Arkansas        53182.0
193           CA  total  2012  37999878.0  California       163707.0

In [38]:
final2012.set_index('state', inplace = True)

In [39]:
density = final2012['population'] / final2012['area (sq. mi)']

In [40]:
density.sort_values(ascending = False, inplace = True)
density.head()

state
District of Columbia    9315.102941
Puerto Rico             1038.846373
New Jersey              1016.710502
Rhode Island             679.808414
Connecticut              647.865260
dtype: float64

In [41]:
density.tail()

state
South Dakota    10.814785
North Dakota     9.919453
Montana          6.837955
Wyoming          5.894886
Alaska           1.112552
dtype: float64